<a href="https://colab.research.google.com/github/mariaclaraseixa/corpo-polimerico-cma/blob/main/resultados_pol%C3%ADmero_cma.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importando a tabela do excel com todos os valores obtido pela maquina durante o ensaio

In [29]:
import pandas as pd

df = pd.read_excel("/Book 1.xlsx", sheet_name="Planilha1")
print(df.head())

   tempo  deslocamento   força     Tensão
0   0.00        0.0000  0.9073  25.984626
1   0.05        0.0005  0.9066  25.964579
2   0.10        0.0032  0.9082  26.010402
3   0.15        0.0078  0.9120  26.119232
4   0.20        0.0132  0.9163  26.242382


# Tenacidade em tração (≈ 3.77 MPa ou MJ/m³)

RESULTADO OBTIDO: aprox 3.77 MPa

Área sob a curva σ × ΔL: 263.97 MPa·mm


In [39]:
# Ordenar os dados pelo deslocamento (por segurança)
df_sorted = df.sort_values("deslocamento")

In [40]:
# extrai os vetores de deslocamento (ΔL em mm) e tensão (σ em MPa)
x = df_sorted["deslocamento"].values  # ΔL em mm
y = df_sorted["Tensão"].values        # σ em MPa

In [41]:
# calculo da área sob a curva tensao x deslocamento usando a regra do trapézio
#  aproxima a integral ∫ σ d(ΔL)
area_sigma_disp = np.trapezoid(y, x)  # resultado em MPa·mm

In [42]:
# comprimento inicial do corpo de prova em mm
L0 = 70.0

# calculo da tenacidade em tração:  U = (1/L0) * ∫ σ d(ΔL)
toughness = area_sigma_disp / L0  # resultado em MPa (que também é MJ/m³)


In [43]:
print("Área sob a curva σ × ΔL (MPa·mm):", area_sigma_disp)

Área sob a curva σ × ΔL (MPa·mm): 263.97233094104547


In [44]:
print("Tenacidade em tração (MPa ou MJ/m³):", toughness)

Tenacidade em tração (MPa ou MJ/m³): 3.7710332991577924


# Módulo da elasticidade (≈ 1.473 GPa)

RESULTADO : 1472.91 MPa (aprox. 1.473 GPa)

In [31]:
mask_elastic = x < 0.3
x_el = x[mask_elastic]
y_el = y[mask_elastic]

# Regressão linear na região elástica: σ ≈ m * ΔL + b
# np.polyfit retorna [inclinação, intercepto]
m, b = np.polyfit(x_el, y_el, 1)

# Inclinação da curva σ x ΔL:
# m = dσ/d(ΔL) em MPa/mm
# Conversão para módulo de elasticidade (σ x ε):
# E = (dσ/dε) = (dσ/dΔL) * (dΔL/dε) = m * L0
E = m * L0  # em MPa


In [32]:
# tensão prevista pela reta elástica em todos os pontos
y_pred = m * x + b

# Desvio entrw valor real e previsto pela reta
deviation = np.abs(y - y_pred) / y_pred

# 1o ponto onde o desvio excede 0,5% (0.005)
threshold = 0.005
idx_yield_candidates = np.where(deviation > threshold)[0]

if len(idx_yield_candidates) > 0:
    idx_yield = idx_yield_candidates[0]
    deltaL_y = x[idx_yield]     # ΔL no escoamento
    sigma_y = y[idx_yield]      # tensão no escoamento
    eps_y = deltaL_y / L0       # deformação no escoamento (adimensional)
else:
    idx_yield = None
    deltaL_y = None
    sigma_y = None
    eps_y = None

In [33]:

print("\nMódulo de Elasticidade")
print(f"Inclinação m (dσ/dΔL)           : {m:.4f} MPa/mm")
print(f"Intercepto b                    : {b:.4f} MPa")
print(f"Módulo de elasticidade E        : {E:.2f} MPa (~ {E/1000:.3f} GPa)")


Módulo de Elasticidade
Inclinação m (dσ/dΔL)           : 21.0416 MPa/mm
Intercepto b                    : 26.0623 MPa
Módulo de elasticidade E        : 1472.91 MPa (~ 1.473 GPa)


# Limite do escoamento (≈ 0.48)

Deformação no escoamento εy ≈ 0.48 %

In [34]:
idx_uts = np.argmax(y)
sigma_uts = y[idx_uts]       # tensão máxima
deltaL_uts = x[idx_uts]      # deslocamento correspondente
eps_uts = deltaL_uts / L0    # deformação correspondente

In [35]:
print("\n Limite de Escoamento (aprox.)")
if idx_yield is not None:
    print(f"Índice do ponto de escoamento   : {idx_yield}")
    print(f"Deslocamento ΔL_y               : {deltaL_y:.4f} mm")
    print(f"Tensão no escoamento σ_y        : {sigma_y:.2f} MPa")
    print(f"Deformação no escoamento ε_y    : {eps_y:.5f} (≈ {eps_y*100:.2f} %)")


 Limite de Escoamento (aprox.)
Índice do ponto de escoamento   : 81
Deslocamento ΔL_y               : 0.3378 mm
Tensão no escoamento σ_y        : 33.00 MPa
Deformação no escoamento ε_y    : 0.00483 (≈ 0.48 %)


# Tensão máxima (≈ 50.35 MPa)

In [36]:
idx_uts = np.argmax(y)
sigma_uts = y[idx_uts]       # tensão máxima
deltaL_uts = x[idx_uts]      # deslocamento correspondente
eps_uts = deltaL_uts / L0    # deformação correspondente

In [37]:
print("\n Tensão Máxima (UTS)")
print(f"Índice do ponto de tensão máx.  : {idx_uts}")
print(f"Deslocamento ΔL_UTS             : {deltaL_uts:.4f} mm")
print(f"Tensão máxima σ_UTS             : {sigma_uts:.2f} MPa")
print(f"Deformação correspondente ε_UTS : {eps_uts:.5f} (≈ {eps_uts*100:.2f} %)")



 Tensão Máxima (UTS)
Índice do ponto de tensão máx.  : 362
Deslocamento ΔL_UTS             : 1.5084 mm
Tensão máxima σ_UTS             : 50.35 MPa
Deformação correspondente ε_UTS : 0.02155 (≈ 2.15 %)
